In [ ]:
from niapy.algorithms.basic import (
    BatAlgorithm,
    FireflyAlgorithm,
    ParticleSwarmAlgorithm
)
from niapy.problems.ackley import Ackley
from niapy.problems.sphere import Sphere
from niapy.runner import Runner
from sklearn.decomposition import PCA
import torch
from torch import nn
from PIL import Image
from matplotlib import pyplot as plt
import os
import pygad
import numpy as np
import pandas as pd
from tools.ml_tools import get_data_loaders, nn_test, nn_train, LSTM
from util.optimization_data import SingleRunData
from tools.optimization_tools import optimization_runner
from meta_ga import solution_to_algorithm_attributes, meta_ga_info

from util.constants import (
    RNG_SEED,
    BATCH_SIZE,
    DATASET_PATH,
    EPOCHS,
    POP_SIZE,
    MAX_ITERS,
    NUM_RUNS,
    OPTIMIZATION_PROBLEM,
    GENE_SPACES,
    POP_DIVERSITY_METRICS,
    INDIV_DIVERSITY_METRICS,
    N_PCA_COMPONENTS,
    LSTM_NUM_LAYERS,
    LSTM_HIDDEN_DIM,
    LSTM_DROPOUT,
    VAL_SIZE,
    TEST_SIZE,
)

meta_ga_info()

execute_training = True

### Optimization

In [ ]:
ga_instance = pygad.load("meta_ga_obj")
print(ga_instance.summary())

In [ ]:

ga_instance.plot_genes(solutions="best")
print(ga_instance.best_solutions[-1])
print(ga_instance.best_solutions_fitness[-1])

algorithms = solution_to_algorithm_attributes(ga_instance.best_solutions[-1], GENE_SPACES)

In [ ]:

test_problem = Ackley(dimension=20)
test_algorithms = [
    #BatAlgorithm(population_size=POP_SIZE, alpha=0.99, gamma=0.9, pulse_rate=0.9, loudness=0.9),
    FireflyAlgorithm(population_size=POP_SIZE, alpha=1.0, beta0=1.0, gamma=0.00),
    ParticleSwarmAlgorithm(population_size=POP_SIZE, c1=0.5, c2=0.5, w=0.75, min_velocity=-10.0, max_velocity=10.0),
]

problem = OPTIMIZATION_PROBLEM

for algorithm in algorithms:
    optimization_runner(
        algorithm=algorithm,
        problem=problem,
        runs=1,
        dataset_path=DATASET_PATH,
        pop_diversity_metrics=POP_DIVERSITY_METRICS,
        indiv_diversity_metrics=INDIV_DIVERSITY_METRICS,
        max_iters=MAX_ITERS,
        rng_seed=RNG_SEED,
        keep_pop_data=True,
        parallel_processing=True,
    )

### Population diversity metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        runs.sort()
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd = SingleRunData.import_from_json(run_path)
        pop_metrics = SingleRunData.import_from_json(run_path).get_pop_diversity_metrics_values(normalize=True)
        ax = pop_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5), fontsize=13)
        ax.set_xlabel(xlabel="Iterations", fontdict={'fontsize':13})
        ax.set_ylabel(ylabel="Value", fontdict={'fontsize':13})

### Best fitness value convergence comparison

In [ ]:
convergence_dict = {}
max_len = 0
algorithms = []
for algorithm in os.listdir(DATASET_PATH):
    algorithms.append(algorithm)
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        runs.sort()
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        convergence = SingleRunData.import_from_json(run_path).get_best_fitness_values(normalize=False)
        if len(convergence) > max_len:
            max_len = len(convergence)
        convergence_dict["_".join([algorithm, problem])] = convergence

for key in convergence_dict:
    convergence = convergence_dict[key]
    print(f"best fitness {key}: {convergence[-1]}")
    convergence = np.append(convergence, [convergence[-1]] * (max_len - len(convergence)))
    convergence_dict[key] = convergence

convergence_dict = pd.DataFrame.from_dict(convergence_dict)
convergence_dict.plot(title=" ".join([*algorithms, problem]), figsize=(20,5))

### Individual diversity metrics comparison

In [ ]:
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        runs.sort()
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd = SingleRunData.import_from_json(run_path)
        indiv_metrics = SingleRunData.import_from_json(run_path).get_indiv_diversity_metrics_values(normalize=True)
        #indiv_metrics.plot(title=" ".join([algorithm, problem]), figsize=(20,5), kind="bar")

        indiv_metrics = indiv_metrics.to_numpy()
        pca = PCA(n_components=N_PCA_COMPONENTS)
        principal_components = pca.fit_transform(indiv_metrics)
        variance = pca.explained_variance_ratio_

        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(principal_components[:,0], principal_components[:,1], principal_components[:,2], color = "green")
        ax.set_xlabel(xlabel="PC1", fontdict={'fontsize':13})
        ax.set_ylabel(ylabel="PC2", fontdict={'fontsize':13})
        ax.set_zlabel(zlabel="PC3", fontdict={'fontsize':13})
        ax.set_xlim((-1.0,1.0))
        ax.set_ylim((1.0,-1.0))
        ax.set_zlim((-1.0,1.0))
        plt.show()

### Diversity metrics euclidean distance

In [ ]:
srd = []
for algorithm in os.listdir(DATASET_PATH):
    for problem in os.listdir(os.path.join(DATASET_PATH, algorithm)):
        runs = os.listdir(os.path.join(DATASET_PATH, algorithm, problem))
        runs.sort()
        run_path = os.path.join(DATASET_PATH, algorithm, problem, runs[0])
        srd.append(SingleRunData.import_from_json(run_path))

print(srd[0].diversity_metrics_euclidean_distance(srd[1]))
print(srd[0].diversity_metrics_euclidean_distance(srd[1], include_fitness_convergence=True))

### LSTM training and test

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
print("CPUs: ", os.cpu_count())

In [ ]:
train_data_loader, val_data_loader, test_data_loader, labels = get_data_loaders(
    dataset_path=DATASET_PATH,
    batch_size=BATCH_SIZE,
    val_size=VAL_SIZE,
    test_size=TEST_SIZE,
    n_pca_components=N_PCA_COMPONENTS,
    problems=[OPTIMIZATION_PROBLEM.name()],
    random_state=RNG_SEED,
)

pop_features, indiv_features, target = next(iter(train_data_loader))
model = LSTM(
    input_dim=np.shape(pop_features)[2],
    aux_input_dim=np.shape(indiv_features)[1],
    num_labels=len(labels),
    hidden_dim=LSTM_HIDDEN_DIM,
    num_layers=LSTM_NUM_LAYERS,
    dropout=LSTM_DROPOUT
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()
model_file_name = f"./lstm_model.pt"

if execute_training:
    model.to(device)
    nn_train(model, train_data_loader, val_data_loader, EPOCHS, loss_fn, optimizer, device, model_file_name, verbal=True)
else:
    model = torch.load(model_file_name, map_location=torch.device(device))
    model.to(device)
    if os.path.exists('loss_plot.png'):
        loss_plot = np.asarray(Image.open('loss_plot.png'))
        plt.axis("off")
        plt.imshow(loss_plot)

In [ ]:
nn_test(model, test_data_loader, device, labels=labels, show_classification_report=True)